# 🔥 Fine-Tuning do GPT-2 com LoRA e Comparação com Fine-Tuning Completo

Este notebook realiza a **adaptação de um modelo GPT-2** para classificação de sentimentos usando o dataset **SST-2 (Stanford Sentiment Treebank)**.  
Serão treinados dois modelos:
1. **GPT-2 com Fine-Tuning Completo**  
2. **GPT-2 com LoRA (Low-Rank Adaptation)**

## 📝 Objetivos
- Avaliar o impacto do LoRA em termos de **eficiência** (parâmetros treináveis e tempo de treino).
- Comparar a **performance** usando **acurácia** e **F1-score**.

## 🚀 Instalação de bibliotecas necessárias


In [1]:
!pip install transformers datasets peft accelerate bitsandbytes

In [2]:
pip install evaluate

In [3]:
pip install --upgrade datasets huggingface-hub fsspec

  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)


In [4]:
pip install --upgrade transformers

## 📚 Importação de Bibliotecas

In [5]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import evaluate
import time
import numpy as np

##📥 Carregando o Dataset SST-2

In [6]:
dataset = load_dataset("glue", "sst2")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


## 🧩 Tokenização

In [7]:
MODEL_NAME = "gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
try:
    tokenizer.padding_side = "left"
except Exception:
    pass

MAX_LEN = 128

def preprocess_function(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_attention_mask=True,
    )

encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["sentence", "idx"])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(encoded_dataset)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})


In [8]:
from transformers import AutoConfig
cfg = AutoConfig.from_pretrained(MODEL_NAME)
print(cfg)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.53.2",
  "use_cache": true,
  "vocab_size": 50257
}



## ⚙️ Funções de Avaliação (Acurácia e F1-score)

In [10]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1_score = f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": acc, "f1": f1_score}

## 🏋️‍♂️ Treinando GPT-2 com Fine-Tuning Completo

In [13]:
model_full = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model_full.config.pad_token_id = tokenizer.eos_token_id

training_args_full = TrainingArguments(
    output_dir="./results_full",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs_full",
    logging_steps=50,
    save_total_limit=1,
    report_to="none"
)

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

start_time_full = time.time()
trainer_full.train()
time_full = time.time() - start_time_full

trainable_params_full = sum(p.numel() for p in model_full.parameters() if p.requires_grad)
print(f"🔍 Parâmetros treináveis (Fine-Tuning Completo): {trainable_params_full}")
print(f"⏱ Tempo de Treinamento (Fine-Tuning Completo): {time_full:.2f} segundos")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-13-1905463601.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_full = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.258100,0.371253,0.863532,0.857485
2,0.177200,0.346341,0.883028,0.886667


🔍 Parâmetros treináveis (Fine-Tuning Completo): 124441344
⏱ Tempo de Treinamento (Fine-Tuning Completo): 3275.93 segundos


# 🔧 Adicionando LoRA ao GPT-2


In [18]:
model_lora = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model_lora.config.pad_token_id = tokenizer.eos_token_id

lora_config = LoraConfig(
    r=4,                # Rank (paper original)
    lora_alpha=16,      # Alpha (paper original)
    lora_dropout=0.05,  # Dropout (paper original)
    target_modules=["c_attn"],
    bias="none",
    task_type="SEQ_CLS"
)

model_lora = get_peft_model(model_lora, lora_config)

trainable_params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
print(f"🔍 Parâmetros treináveis (LoRA): {trainable_params_lora}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔍 Parâmetros treináveis (LoRA): 148992


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [19]:
def print_trainable_parameters(model):
    trainable = 0
    total = 0
    for _, param in model.named_parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Parâmetros treináveis: {trainable} / {total} "
          f"({100 * trainable / total:.2f}%)")

print_trainable_parameters(model_lora)

Parâmetros treináveis: 148992 / 124590336 (0.12%)


## 🏋️‍♂️ Treinando GPT-2 com LoRA

In [20]:
training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs_lora",
    logging_steps=50,
    save_total_limit=1,
    report_to="none"
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

start_time_lora = time.time()
trainer_lora.train()
time_lora = time.time() - start_time_lora

print(f"🔍 Parâmetros treináveis (LoRA): {trainable_params_lora}")
print(f"⏱ Tempo de Treinamento (LoRA): {time_lora:.2f} segundos")


/tmp/ipython-input-20-1763312014.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.356900,0.332402,0.852064,0.851894
2,0.309700,0.323243,0.862385,0.866962


🔍 Parâmetros treináveis (LoRA): 148992
⏱ Tempo de Treinamento (LoRA): 2312.13 segundos


## 📊 Comparação Final


In [21]:
results_full = trainer_full.evaluate()
results_lora = trainer_lora.evaluate()

print("=== Resultados Fine-Tuning Completo ===")
print(results_full)

print("\n=== Resultados LoRA ===")
print(results_lora)

print(f"\n🔍 Parâmetros Fine-Tuning Completo: {trainable_params_full}")
print(f"🔍 Parâmetros LoRA: {trainable_params_lora}")
print(f"⏱ Tempo FT Completo: {time_full:.2f} s")
print(f"⏱ Tempo LoRA: {time_lora:.2f} s")

=== Resultados Fine-Tuning Completo ===
{'eval_loss': 0.3463413715362549, 'eval_accuracy': 0.8830275229357798, 'eval_f1': 0.8866666666666667, 'eval_runtime': 6.5323, 'eval_samples_per_second': 133.491, 'eval_steps_per_second': 8.42, 'epoch': 2.0}

=== Resultados LoRA ===
{'eval_loss': 0.3232429623603821, 'eval_accuracy': 0.8623853211009175, 'eval_f1': 0.8669623059866962, 'eval_runtime': 6.7958, 'eval_samples_per_second': 128.314, 'eval_steps_per_second': 8.093, 'epoch': 2.0}

🔍 Parâmetros Fine-Tuning Completo: 124441344
🔍 Parâmetros LoRA: 148992
⏱ Tempo FT Completo: 3275.93 s
⏱ Tempo LoRA: 2312.13 s
